In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import time

In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from module.prepare_data import *

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
# def _make_lag_roll(base_test, target, shift_day, roll_wind):
#     # target, shift_day, roll_wind = LAG_DAY[0],LAG_DAY[1],LAG_DAY[2]
#     lag_df = base_test[['id','d',target]]
#     col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
#     lag_df[col_name] = lag_df.groupby(['id'])[target].rolling(roll_wind).parallel_apply(np.mean).reset_index(0, drop=True)
#     print(lag_df[col_name].shape, lag_df.groupby(['id'])[col_name].transform(lambda x: x.shift(shift_day)).shape)
    
#     lag_df[col_name] = lag_df.groupby(['id'])[col_name].transform(lambda x: x.shift(shift_day))

#     # grid_df[f'rolling{i}_shift(shift_day)_fft_diff_amp_top{top_no}'] = grid_df.groupby(["id"])["sales_diff"].rolling(
#     #     i).parallel_apply(fft_peak).reset_index(0, drop=True)
#     # grid_df[f'rolling{i}_shift(shift_day)_fft_diff_amp_top{top_no}'] = grid_df.groupby(["id"])[
#     #     f'rolling{i}_shift(shift_day)_fft_diff_amp_top{top_no}'].transform(lambda x: x.shift(shift_day))
#     return lag_df[[col_name]]

In [ ]:
if not os.path.exists(TEMP_FEATURE_PKL):
    train_df = pd.read_csv(f'{ORI_CSV_PATH}/sales_train_evaluation.csv')
    prices_df = pd.read_csv(f'{ORI_CSV_PATH}/sell_prices.csv')
    calendar_df = pd.read_csv(f'{ORI_CSV_PATH}/calendar.csv')
    try:
        if not os.path.exists(BASE_PATH):
            os.makedirs(BASE_PATH)
    except OSError:
        print("Creation of the directory %s failed" % BASE_PATH)
    else:
        print("Successfully created the directory %s" % BASE_PATH)

    grid_df = extract_features(train_df, prices_df, calendar_df, target=TARGET, nan_mask_d=1913+28)
    
    grid_df['item_id'] = grid_df['item_id'].astype('category')
    grid_df['dept_id'] = grid_df['dept_id'].astype('category')
    grid_df['cat_id'] = grid_df['cat_id'].astype('category')
#     grid_df['item_id'] = grid_df['item_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['dept_id'] = grid_df['dept_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['cat_id'] = grid_df['cat_id'].replace({'HOBBIES': 0, 'HOUSEHOLD': 1, 'FOODS': 2}).astype('category')
#     for col in ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
#         grid_df[col] = grid_df[col].replace(
#             dict(zip(grid_df[col].unique(), np.arange(grid_df[col].unique().shape[0])))).astype('category')
    grid_df = reduce_mem_usage(grid_df)
    grid_df.to_pickle(TEMP_FEATURE_PKL)
    del grid_df

Successfully created the directory ../cache/ver620
melt_train_df 32.51199436187744
extract_price_features 26.01192617416382
extract_calendar_features 60.52012538909912


In [ ]:
grid_df = pd.read_pickle(TEMP_FEATURE_PKL)
grid_df['groups'] = grid_df['tm_y'].astype(str) + '_' + (grid_df['tm_m']//3).astype(str)
grid_df.to_pickle(TEMP_FEATURE_PKL)
del grid_df

In [9]:
useful_cols = dict(zip(STORES_IDS, [M5_FEATURES]*len(STORES_IDS)))

In [10]:
from module.lgbm_baseline import *

In [11]:
history_df = train_evaluate_model(useful_cols, TARGET, BASE_PATH)#stores_ids=['CA_1']
print(history_df.rmse_trn.mean(), history_df.rmse_val.mean(), history_df.rmse_diff.mean())

Train CA_1


EOFError: Ran out of input